In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xarray as xr
from dateutil.relativedelta import relativedelta

from my_functions import read_obsfcstana_extend_datetime
from my_functions import read_obsfcstana

In [ ]:
# expt_name = 'DAv7_M36_ASCAT_type_13_no_catdef_fp', 'DAv7_M36_ASCAT_type_2_fp_precip', 'DAv7_M36_ASCAT_type_13_test_catdef'
expt_name = 'OLv7_M36_ASCAT_type_13_comb_fp'

start_date = datetime(2015, 4, 1)
end_date = datetime(2021, 4, 1)

start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

In [ ]:
# Read in the cross mask tile information
npzfile = np.load(f'{expt_name}_{start_date_str}_{end_date_str}_obsfcstana_stats.npz')
unique_tilenum = npzfile['unique_tilenum']
num_obs_smap_cm = npzfile['num_obs_smap_cm']
num_obs_ascat_cm = npzfile['num_obs_ascat_cm']

# Find the tiles with both ASCAT and SMAP observations
tilenum_both = unique_tilenum[(num_obs_smap_cm > 0) & (num_obs_ascat_cm > 0)]

In [ ]:
# Calculate the daily statistics in observation space

# Define the path directory
path_dir = f'/discover/nobackup/amfox/Experiments/{expt_name}/{expt_name}/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg'

# Define the common file name start
file_name_start = f'{expt_name}.ens_avg.ldas_ObsFcstAna.'

# Define the print flag
printflag = False

# Initialize lists to store the mean values for each variable and the dates
dates_list = []

ascat_obs_obs_count_list = []
ascat_obs_obs_mean_list = []
ascat_obs_fcst_mean_list = []
ascat_obs_ana_mean_list = []
ascat_omf_mean_list = []
ascat_oma_mean_list = []
ascat_omf_max_list = []
ascat_omf_std_list = []
ascat_oma_std_list = []

smap_obs_obs_count_list = []
smap_obs_obs_mean_list = []
smap_obs_fcst_mean_list = []
smap_obs_ana_mean_list = []
smap_omf_mean_list = []
smap_oma_mean_list = []
smap_omf_max_list = []
smap_omf_std_list = []
smap_oma_std_list = []

# Define the start and end dates
# start_date = datetime.strptime('20150401', '%Y%m%d')
# end_date = datetime.strptime('20210331', '%Y%m%d')

# Loop over the dates
current_date = start_date
while current_date <= end_date:
    # Define the file name for the current date
    file_name = file_name_start + current_date.strftime('%Y%m%d')
    if file_name[-4:] == '0401':
        print('file_name = ', file_name)
    
    # Call the read_obsfcstana function for the current file
    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana(path_dir, file_name, printflag)

    # Convert the lists to numpy arrays
    a_obs_obs = np.array(obs_obs[(obs_species > 4) & np.isin(obs_tilenum, tilenum_both)])
    a_obs_fcst = np.array(obs_fcst[(obs_species > 4) & np.isin(obs_tilenum,tilenum_both)])
    a_obs_ana = np.array(obs_ana[(obs_species > 4) & np.isin(obs_tilenum, tilenum_both)])
    
    # Calculate the mean values for the variables
    obs_obs_count = len(a_obs_obs)
    obs_obs_mean = np.mean(a_obs_obs)
    obs_fcst_mean = np.mean(a_obs_fcst)
    obs_ana_mean = np.mean(a_obs_ana)
    omf_mean = np.mean(a_obs_obs - a_obs_fcst)
    oma_mean = np.mean(a_obs_obs - a_obs_ana)
    # Calculate the maximum absolute difference between obs_obs and obs_fcst
    if a_obs_fcst.size > 0 and a_obs_obs.size > 0 and a_obs_fcst.shape == a_obs_obs.shape:
        omf_max = np.max(abs(a_obs_obs - a_obs_fcst))
    else:
        omf_max = np.nan
        print('Current date = ', current_date)
    omf_std = np.std(a_obs_obs - a_obs_fcst)
    oma_std = np.std(a_obs_obs - a_obs_ana)

    # Append the mean values to the lists
    ascat_obs_obs_count_list.append(obs_obs_count)
    ascat_obs_obs_mean_list.append(obs_obs_mean)
    ascat_obs_fcst_mean_list.append(obs_fcst_mean)
    ascat_obs_ana_mean_list.append(obs_ana_mean)
    ascat_omf_mean_list.append(omf_mean)
    ascat_oma_mean_list.append(oma_mean)
    ascat_omf_max_list.append(omf_max)
    ascat_omf_std_list.append(omf_std)
    ascat_oma_std_list.append(oma_std)
    
    # Repeat for the SMAP obs
    # Convert the lists to numpy arrays
    s_obs_obs = np.array(obs_obs[(obs_species < 5) & np.isin(obs_tilenum, tilenum_both)])
    s_obs_fcst = np.array(obs_fcst[(obs_species < 5) & np.isnin(obs_tilenum, tilenum_both)])
    s_obs_ana = np.array(obs_ana[(obs_species < 5) & np.isin(obs_tilenum, tilenum_both)])
    
    # Calculate the mean values for the variables
    obs_obs_count = len(s_obs_obs)
    obs_obs_mean = np.mean(s_obs_obs)
    obs_fcst_mean = np.mean(s_obs_fcst)
    obs_ana_mean = np.mean(s_obs_ana)
    omf_mean = np.mean(s_obs_obs - s_obs_fcst)
    oma_mean = np.mean(s_obs_obs - s_obs_ana)
    # Calculate the maximum absolute difference between obs_obs and obs_fcst
    if s_obs_fcst.size > 0 and s_obs_obs.size > 0 and s_obs_fcst.shape == s_obs_obs.shape:
        omf_max = np.max(abs(s_obs_obs - s_obs_fcst))
    else:
        omf_max = np.nan
        print('Current date = ', current_date)
    omf_std = np.std(s_obs_obs - s_obs_fcst)
    oma_std = np.std(s_obs_obs - s_obs_ana)

    # Append the mean values to the lists
    smap_obs_obs_count_list.append(obs_obs_count)
    smap_obs_obs_mean_list.append(obs_obs_mean)
    smap_obs_fcst_mean_list.append(obs_fcst_mean)
    smap_obs_ana_mean_list.append(obs_ana_mean)
    smap_omf_mean_list.append(omf_mean)
    smap_oma_mean_list.append(oma_mean)
    smap_omf_max_list.append(omf_max)
    smap_omf_std_list.append(omf_std)
    smap_oma_std_list.append(oma_std)    
        
    # Append the current date to the dates list
    dates_list.append(current_date.strftime('%Y%m%d'))

    # Increment the current date by one day
    current_date += timedelta(days=1)



In [ ]:
np.savez(f'{expt_name}_{start_date_str}_{end_date_str}_OmF_ts_cross_masked.npz',
         dates_list=dates_list,
         
         ascat_obs_obs_count_list=ascat_obs_obs_count_list,
         ascat_obs_obs_mean_list=ascat_obs_obs_mean_list,
         ascat_obs_fcst_mean_list=ascat_obs_fcst_mean_list,
         ascat_obs_ana_mean_list=ascat_obs_ana_mean_list,
         ascat_omf_mean_list=ascat_omf_mean_list,
         ascat_oma_mean_list=ascat_oma_mean_list,
         ascat_omf_max_list=ascat_omf_max_list,
         ascat_omf_std_list=ascat_omf_std_list,
         ascat_oma_std_list=ascat_oma_std_list,
        
         smap_obs_obs_count_list=smap_obs_obs_count_list,
         smap_obs_obs_mean_list=smap_obs_obs_mean_list,
         smap_obs_fcst_mean_list=smap_obs_fcst_mean_list,
         smap_obs_ana_mean_list=smap_obs_ana_mean_list,
         smap_omf_mean_list=smap_omf_mean_list,
         smap_oma_mean_list=smap_oma_mean_list,
         smap_omf_max_list=smap_omf_max_list,
         smap_omf_std_list=smap_omf_std_list,
         smap_oma_std_list=smap_oma_std_list) 